# Assignment A3.1

**Intuitive exploration, choosing which columns to drop based on amount of unique values, conceptual relevance and assumptions made on impact on classification question**

Description of the Dataset (From source on Kaggle):
>"I concatenated historical loans from both Prosper and Lending Club 2013 - 2018. Currently only the summary of the loan (terms, origination date, loan amount, status, etc) are up but detailed lender data will come soon. The columns are matched up as accurately as possible but there are estimated columns, see below for more info."

Using your coding skills:

   - Perform a classification analysis to separate defaulted loans from the good loans (Make your own assumptions). (Logistic Regression may work in this case) 
   - Make some visualizations that showcase which are the more important features. (Decision trees could be a good idea for this) 
   - The data analysis should also help the viewer to understand the data. (Storytell)

In [5]:
import pandas as pd
import numpy as mp
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

Loading data for inspection and cleaning. Reminder: the warning raised is because 'listing_title' has some nans in it. This should be processed before continuing in the analysis, maybe filled with 'unknown'

In [2]:
pre_data = pd.read_csv('data/Master_Loan_Summary.csv')

/home/khaerus/anaconda3/envs/DataPy/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
pre_data.head()

,loan_number,amount_borrowed,term,borrower_rate,installment,grade,origination_date,listing_title,principal_balance,principal_paid,interest_paid,late_fees_paid,debt_sale_proceeds_received,last_payment_date,next_payment_due_date,days_past_due,loan_status_description,data_source
0,10149342,27050.0,36,0.1099,885.46,B,2013-12-01T00:00,debt_consolidation,0.0,27050.0,4702.53,0.0,0.0,2016-07-01T00:00,2016-08-01T00:00,0,COMPLETED,Lending Club
1,10149488,4800.0,36,0.1099,157.13,B,2013-12-01T00:00,home_improvement,0.0,4800.0,357.52,0.0,0.0,2014-09-01T00:00,2014-10-01T00:00,0,COMPLETED,Lending Club
2,10148122,12000.0,36,0.0762,373.94,A,2013-12-01T00:00,debt_consolidation,0.0,12000.0,1397.54,0.0,0.0,2016-06-01T00:00,2016-07-01T00:00,0,COMPLETED,Lending Club
3,10119623,12000.0,36,0.1199,398.52,B,2013-12-01T00:00,debt_consolidation,0.0,12000.0,2346.48,0.0,0.0,2017-01-01T00:00,2017-02-01T00:00,0,COMPLETED,Lending Club
4,10159498,12000.0,36,0.0662,368.45,A,2013-12-01T00:00,debt_consolidation,0.0,12000.0,1263.95,0.0,0.0,2017-01-01T00:00,2017-02-01T00:00,0,COMPLETED,Lending Club


In [4]:
pre_data.columns.values

array(['loan_number', 'amount_borrowed', 'term', 'borrower_rate',
       'installment', 'grade', 'origination_date', 'listing_title',
       'principal_balance', 'principal_paid', 'interest_paid',
       'late_fees_paid', 'debt_sale_proceeds_received',
       'last_payment_date', 'next_payment_due_date', 'days_past_due',
       'loan_status_description', 'data_source'], dtype=object)

I think I'm choosing to drop 'loan_number', 'origination_date', 'principal_balance', 'principal_paid', 'interest_paid', 'last_payment_date', 'next_payment_due_date', 'days_past_due', 'data_source' as per intuition I consider these columns to be non-relevant for the research question, either because they just serve to provide identification, are centered on dates, events or repetitive information that could be noiseful. There **will** be noise in the rest of columns but I consider them to have more important information in them, given their definitions, I'll use decision trees to evaluate their feature importance ranking.

In [4]:
len(pre_data['days_past_due'].unique())

1635

In [7]:
columns = ['loan_number', 'origination_date', 'principal_balance', 'principal_paid', 'interest_paid', 'last_payment_date', 'next_payment_due_date', 'days_past_due', 'data_source' ]
data = pre_data.drop(columns, axis=1)

In [8]:
data.head()

,amount_borrowed,term,borrower_rate,installment,grade,listing_title,late_fees_paid,debt_sale_proceeds_received,loan_status_description
0,27050.0,36,0.1099,885.46,B,debt_consolidation,0.0,0.0,COMPLETED
1,4800.0,36,0.1099,157.13,B,home_improvement,0.0,0.0,COMPLETED
2,12000.0,36,0.0762,373.94,A,debt_consolidation,0.0,0.0,COMPLETED
3,12000.0,36,0.1199,398.52,B,debt_consolidation,0.0,0.0,COMPLETED
4,12000.0,36,0.0662,368.45,A,debt_consolidation,0.0,0.0,COMPLETED


This is an interesting feature that could be important, I'm keeping it and assuming it's in months for later interpretation

In [7]:
data['term'].unique()

array([36, 60, 12])

The loan status description could work as target variable using a dictionary to map 'completed' and 'current' loans as '1' (positive) and 'chargeoff', 'defaulted' and 'cancelled' as '0' (negative). On the basis that completed and current loans belong to clients that were good enough to payoff their loans or that haven't completed them but are punctual payers. All other categories (chargeoff, defaulted or cancelled) are bad for business.

In [8]:
data['loan_status_description'].unique()

array(['COMPLETED', 'CHARGEOFF', 'CURRENT', 'DEFAULTED', 'CANCELLED'],
      dtype=object)

Creating a dictionary for the mapping task ('binary')

In [9]:
status = data['loan_status_description'].unique()
status_binary = [1,0,1,0,0]
status_binary_map = dict(zip(status,status_binary))

In [10]:
status_binary_map

{'COMPLETED': 1, 'CHARGEOFF': 0, 'CURRENT': 1, 'DEFAULTED': 0, 'CANCELLED': 0}

Creating a dictionary for the mapping task ('good or bad'), for plotting purposes

In [11]:
status_text_map = {1:'good', 0:'bad'}

In [12]:
test = data['loan_status_description'].map(status_binary_map)
test.sum()/test.count()*100

88.85399211031371

**Interesting! Most loans (88.85%) are good loans!** That means it must be hard to pick up the bad ones...What happens if I drop the ones with 'CURRENT'? that are loans that could turn bad in the future?

In [12]:
mask = data['loan_status_description'] == 'CURRENT'
len(data[mask])/len(data)*100

48.05042943906153

Spoiler alert! This is gonna be an interesting choice to make, almost half of loans are current! The plot thickens, maybe I could add a filter or a side analysis to check how far on their payment schedule they are and see if it really is bad to take them into account. 

In [13]:
test_frame = data[~mask]
test = test_frame['loan_status_description'].map(status_binary_map)
test.sum()/test.count()*100

78.54456202556732

After dropping loans with status as current, the fraction of 'good' loans drops only 10%...I think I'll maintain the assumption that loans marked as 'CURRENT' are good for business loans, If later on the analysis this seems to be a risky assumption, I'll change my mind.

In [14]:
data['status_binary'] = data['loan_status_description'].map(status_binary_map)
data['status_simplified'] = data['status_binary'].map(status_text_map)

Listing title has some pretty interesting information, it may serve to explore if there is any clustering around loan grade for certain motives

In [15]:
data.listing_title.unique()

array(['debt_consolidation', 'home_improvement', 'credit_card', 'other',
       'major_purchase', 'small_business', 'house', 'moving', 'medical',
       'car', 'vacation', 'renewable_energy', 'wedding', 'educational',
       nan], dtype=object)

In [16]:
data.head()

,amount_borrowed,term,borrower_rate,installment,grade,listing_title,late_fees_paid,debt_sale_proceeds_received,loan_status_description,status_binary,status_simplified
0,27050.0,36,0.1099,885.46,B,debt_consolidation,0.0,0.0,COMPLETED,1,good
1,4800.0,36,0.1099,157.13,B,home_improvement,0.0,0.0,COMPLETED,1,good
2,12000.0,36,0.0762,373.94,A,debt_consolidation,0.0,0.0,COMPLETED,1,good
3,12000.0,36,0.1199,398.52,B,debt_consolidation,0.0,0.0,COMPLETED,1,good
4,12000.0,36,0.0662,368.45,A,debt_consolidation,0.0,0.0,COMPLETED,1,good


In [18]:
data['listing_title'] = data['listing_title'].fillna(value='unknown')

In [19]:
data.listing_title.unique()

array(['debt_consolidation', 'home_improvement', 'credit_card', 'other',
       'major_purchase', 'small_business', 'house', 'moving', 'medical',
       'car', 'vacation', 'renewable_energy', 'wedding', 'educational',
       'unknown'], dtype=object)

In [23]:
data.to_csv('data/cleaned_full_set.csv', index=False)

Size reduced in half! Onto next notebook!

----------------------------------------------------------------------------------------------------

Saving imports for later sections when it makes more sense after doing an EDA. Probably moving onto a new notebook.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from adspy_shared_utilities import (
plot_class_regions_for_classifier_subplot)
from sklearn.dummy import DummyClassifier
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier